In [17]:
#Libraries
import pandas as pd
import numpy as np
import os
import random
import cv2

#Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve
)

# Viz
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns

#Image
import PIL.Image
from PIL import Image
from PIL import ImageOps
from IPython.display import display

#Tensor flow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#Load data from hugging face
from datasets import load_dataset
import requests
#from zipfile import ZipFile

In [25]:
#Uploading the dataset
from google.colab import files
uploaded = files.upload()

Saving Dataset_BUSI_with_GT.zip to Dataset_BUSI_with_GT (1).zip


In [29]:
#unzipping the file
import zipfile
import os

zip_path = "Dataset_BUSI_with_GT.zip"
extract_to = "/content"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

#confirm location
extracted_path = os.path.join(extract_to, "Dataset_BUSI_with_GT")
print(extracted_path)

/content/Dataset_BUSI_with_GT


In [30]:
os.listdir(extracted_path)

['benign', 'normal', 'malignant']